# Vietoris-Rips Filtration Demo

This notebook demonstrates persistent homology by:
1. Using 4 hardcoded points arranged in a quadrilateral (to demonstrate H1 features with gradual edge formation)
2. Computing persistent homology using ripser
3. Visualizing the Vietoris-Rips filtration at different radius values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Polygon
from scipy.spatial.distance import pdist, squareform
from ripser import ripser
from persim import plot_diagrams
from itertools import combinations

## Define Points

In [ ]:
# Hardcoded 4 points arranged in a quadrilateral to show H1 features
# Points positioned to create gradual edge formation: vertical edges first, then horizontal, then diagonals
points = np.array([
    [-2.0, -1.0],    # p1 (bottom-left)
    [-2.0, 2.0],     # p2 (top-left)
    [3.5, -1.5],     # p3 (bottom-right) - moved right to create different horizontal edge distances
    [3.0, 2.5]       # p4 (top-right)
])

print("Hardcoded points (quadrilateral configuration):")
for i, p in enumerate(points):
    print(f"Point p{i+1}: ({p[0]:.2f}, {p[1]:.2f})")

# Compute pairwise distances
distances = squareform(pdist(points))
print(f"\nDistance matrix shape: {distances.shape}")
print(f"\nPairwise distances:")
for i in range(len(points)):
    for j in range(i+1, len(points)):
        print(f"  d(p{i+1}, p{j+1}) = {distances[i,j]:.2f}")

## Compute Persistent Homology

In [ ]:
# Compute persistent homology using Vietoris-Rips filtration
result = ripser(points, maxdim=1)

# Extract persistence diagrams
diagrams = result['dgms']

print("H0 (connected components):")
print(diagrams[0])
print("\nH1 (loops/holes):")
print(diagrams[1])

## Visualize Persistence Diagrams

In [ ]:
# Plot persistence diagrams
plot_diagrams(diagrams)
plt.suptitle('Persistence Diagrams', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Visualize Vietoris-Rips Filtration

Show how the simplicial complex evolves as the radius parameter increases.

In [ ]:
def plot_filtration_step(points, radius, ax, title):
    """
    Plot the Vietoris-Rips complex at a given radius.
    
    Args:
        points: Array of shape (n, 2) containing point coordinates
        radius: The current radius for the filtration
        ax: Matplotlib axis to plot on
        title: Title for the subplot
    """
    n = len(points)
    distances = squareform(pdist(points))
    
    # Find edges (pairs of points within distance r)
    edges = []
    for i in range(n):
        for j in range(i+1, n):
            if distances[i, j] <= radius:  # Edge exists when circle of radius r reaches the other point
                edges.append((i, j))
    
    # Find triangles (triples where all three pairwise distances ≤ radius)
    triangles = []
    for i, j, k in combinations(range(n), 3):
        if (distances[i, j] <= radius and 
            distances[i, k] <= radius and 
            distances[j, k] <= radius):
            triangles.append((i, j, k))
    
    # Plot triangles (filled)
    for tri in triangles:
        triangle_points = points[list(tri)]
        polygon = Polygon(triangle_points, alpha=0.3, facecolor='lightgray', edgecolor='black', linewidth=1)
        ax.add_patch(polygon)
    
    # Plot edges
    for i, j in edges:
        ax.plot([points[i, 0], points[j, 0]], 
               [points[i, 1], points[j, 1]], 
               'k-', linewidth=1.5, zorder=1)
    
    # Plot points
    ax.scatter(points[:, 0], points[:, 1], c='blue', s=100, zorder=2, edgecolors='black')
    
    # Add labels - position below for bottom points (p1, p3), above for top points (p2, p4)
    labels = [f'p{i+1}' for i in range(n)]
    for i, (x, y) in enumerate(points):
        # Place label below if y < 0, above otherwise
        offset = -0.5 if y < 0 else 0.4
        ax.text(x, y + offset, labels[i], ha='center', fontsize=10)
    
    ax.set_xlim(-4, 5)
    ax.set_ylim(-3, 4)
    ax.set_aspect('equal')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

In [ ]:
# Choose radius values to show gradual edge addition
# Distances: d(p1,p2)=3, d(p3,p4)=4, d(p2,p4)≈5.0, d(p1,p3)≈5.5, d(p1,p4)≈6.1, d(p2,p3)≈6.5
radius_values = [0, 3, 4, 5.1, 5.6, 6.15]

# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, r in enumerate(radius_values):
    plot_filtration_step(points, r, axes[idx], f'r = {r:.2f}')

plt.tight_layout()
plt.savefig('final_paper/images/filtration_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFiltration visualization saved as 'final_paper/images/filtration_visualization.png'")